## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [4]:
message = "Latest AI Agent Framework in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent systems.

**LangChain** is a modular framework that enables developers to build complex AI applications by chaining language models, external tools, and custom memory. It supports integration with multiple large language models (LLMs) and provides tools for API interactions, semantic search, and prompt engineering. Ideal for conversational AI and automated research assistants. ([linkedin.com](https://www.linkedin.com/pulse/top-5-frameworks-building-ai-agents-2025-sahil-malhotra-wmisc?utm_source=openai))

**LangGraph**, an extension of LangChain, focuses on stateful, multi-agent systems using a graph-based architecture. It facilitates complex workflows requiring deterministic control, retries, and branching logic, making it suitable for interactive storytelling and strategic simulations. ([linkedin.com](https://www.linkedin.com/pulse/top-5-frameworks-building-ai-agents-2025-sahil-malhotra-wmisc?utm_source=openai))

**AutoGen**, developed by Microsoft, specializes in orchestrating multiple AI agents in a collaborative, asynchronous environment. It allows for role-based agent assignments and supports multi-agent orchestration, making it ideal for enterprise-level applications needing event-driven workflows. ([linkedin.com](https://www.linkedin.com/pulse/top-5-frameworks-building-ai-agents-2025-sahil-malhotra-wmisc?utm_source=openai))

**CrewAI** adopts a role-based agent collaboration approach, facilitating the creation of specialized agents that collaborate on complex projects akin to a team environment. It offers dynamic task planning and real-time performance monitoring, suitable for complex projects requiring teamwork among agents. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-june-2025-comprehensive-overview-chadi-abi-fadel-wcu5c?utm_source=openai))

**Amazon Bedrock AgentCore** is a platform introduced by AWS to support enterprise-scale AI agent development and deployment. It offers tools for creating smarter, safer, and more intuitive AI agents, leveraging AWS’s infrastructure expertise. ([techradar.com](https://www.techradar.com/pro/we-want-aws-to-be-the-place-where-everyone-runs-enterprise-ai-agents-the-agentic-era-is-here-for-your-business-so-be-prepared-for-the-new-age?utm_source=openai))

**OpenAI Operator** is an AI agent capable of autonomously performing tasks through web browser interactions, including filling forms, placing online orders, and scheduling appointments. It uses OpenAI's advanced models to expand practical automation capabilities for users in daily activities. ([en.wikipedia.org](https://en.wikipedia.org/wiki/OpenAI_Operator?utm_source=openai))

These frameworks represent the forefront of AI agent development in 2025, each catering to different needs and applications in the field. 

### We will now use Structured Outputs, and include a description of the fields

In [5]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [6]:
message = "Latest AI Agent Framework in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find the most recent developments and frameworks in AI agents as of 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To discover advancements and trends in AI agents, including new technologies and methodologies.', query='AI agent technologies 2025'), WebSearchItem(reason='To look for specific examples, case studies, or reviews of AI agent frameworks released or updated in 2025.', query='reviews of AI agent frameworks 2025')]


In [7]:
import AIRPALibrary
ailabs = AIRPALibrary.AIRPALibrary()

In [15]:
@function_tool
def send_email(subject: str, body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    
    email_to = 'muhammad.fakhrurrozi@ai.astra.co.id'
    email_from = 'elvino.dwisaputra@ai.astra.co.id'
    ailabs.send_email(subject=subject, content=f"""{body}""", to=email_to, sender=email_from)
    return {"status": "success"}

In [16]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'body': {'title': 'Body', 'type': 'string'}}, 'required': ['subject', 'body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020F4E898AE0>, strict_json_schema=True, is_enabled=True)

In [17]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



In [18]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [19]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and email it

In [20]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

### Showtime!

In [21]:
query = "Latest AI Agent Framework in 2025"

with trace("Research trace"):
    print("Starting research...")
    
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)
    print("Done")

Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
{
  "message": "Email berhasil dikirim!",
  "success": true
}

Email sent
Done
